In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [5]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv("./home_sales_revised.csv", sep=",", header=True, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)
df.createOrReplaceTempView('home_sales')
df.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import year, avg, format_number

# Filter by four-bedroom houses
four_bedroom_houses = df.filter(df['bedrooms'] == 4)

# Extract the year of the sale date and calculate the average price per year
average_price_per_year = four_bedroom_houses.groupBy(year('date').alias('year')) \
                            .agg(format_number(avg('price'), 2).alias('average_price')) \
                            .orderBy('year')
# Show results
average_price_per_year.show(15) 

+----+-------------+
|year|average_price|
+----+-------------+
|2019|   300,263.70|
|2020|   298,353.78|
|2021|   301,819.44|
|2022|   296,363.88|
+----+-------------+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# Filter by three-bedroom and three-bathroom houses
three_bedroom_and_bathroom_houses = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3))

# Group by the year the home was built and calculate the average price
average_price_per_build_year = three_bedroom_and_bathroom_houses.groupBy('date_built') \
                            .agg(format_number(round(mean('price'), 2), 2).alias('average_price')) \
                            .orderBy('date_built')
# Show results
average_price_per_build_year.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|   292,859.62|
|      2011|   291,117.47|
|      2012|   293,683.19|
|      2013|   295,962.27|
|      2014|   290,852.27|
|      2015|   288,770.30|
|      2016|   290,555.07|
|      2017|   292,676.79|
+----------+-------------+



In [14]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# Filtrar por casas de tres dormitorios, tres baños, dos pisos y mayor o igual a 2000 pies cuadrados
filtered_houses = df.filter(
    (df['bedrooms'] == 3) & 
    (df['bathrooms'] == 3) & 
    (df['floors'] == 2) & 
    (df['sqft_living'] >= 2000)
)

# Group by the year the home was built and calculate the average price
average_price_per_year_construction = filtered_houses.groupBy('date_built') \
                            .agg(format_number(round(mean('price'), 2), 2).alias('average_price')) \
                            .orderBy('date_built')
    
average_price_per_year_construction.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|   285,010.22|
|      2011|   276,553.81|
|      2012|   307,539.97|
|      2013|   303,676.79|
|      2014|   298,264.72|
|      2015|   297,609.97|
|      2016|   293,965.10|
|      2017|   280,317.58|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

home_filter = df.filter(df['price'] >= 350000)

# Agrupar por la calificación de "view" y calcular el precio promedio
average_price_per_view = home_filter.groupBy('view') \
                            .agg(format_number(avg('price'), 2).alias('average_price')) \
                            .orderBy('view')

average_price_per_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|   403,848.51|
|   1|   401,044.25|
|  10|   401,868.43|
| 100| 1,026,669.50|
|  11|   399,548.12|
|  12|   401,501.32|
|  13|   398,917.98|
|  14|   398,570.03|
|  15|   404,673.30|
|  16|   399,586.53|
|  17|   398,474.49|
|  18|   399,332.91|
|  19|   398,953.17|
|   2|   397,389.25|
|  20|   399,522.81|
|  21|   399,758.88|
|  22|   402,022.68|
|  23|   403,411.92|
|  24|   400,284.92|
|  25|   401,298.69|
+----+-------------+
only showing top 20 rows

--- 0.20149922370910645 seconds ---


In [9]:
# 7. Cache the temporary table home_sales.
spark.catalog.cacheTable('home_sales')

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

filter_house_using_cache = spark.sql("""
    SELECT view, FORMAT_NUMBER(AVG(price), 2) as average_price 
    FROM home_sales 
    WHERE price >= 350000 
    GROUP BY view 
    ORDER BY view
""")
filter_house_using_cache.show()

print("--- %s seconds ---" % (time.time() - start_time))

# Delete table from cache
# spark.catalog.uncacheTable('home_sales')

+----+-------------+
|view|average_price|
+----+-------------+
|   0|   403,848.51|
|   1|   401,044.25|
|  10|   401,868.43|
| 100| 1,026,669.50|
|  11|   399,548.12|
|  12|   401,501.32|
|  13|   398,917.98|
|  14|   398,570.03|
|  15|   404,673.30|
|  16|   399,586.53|
|  17|   398,474.49|
|  18|   399,332.91|
|  19|   398,953.17|
|   2|   397,389.25|
|  20|   399,522.81|
|  21|   399,758.88|
|  22|   402,022.68|
|  23|   403,411.92|
|  24|   400,284.92|
|  25|   401,298.69|
+----+-------------+
only showing top 20 rows

--- 0.10400032997131348 seconds ---


In [24]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
import os
output_path = "./save_data"
df.write.partitionBy("date_built").parquet(output_path, mode='overwrite')

In [27]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet('p_sales')

In [29]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_sales')

In [32]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
start_time = time.time()

filter_house_using_parquet = spark.sql("""
    SELECT view, FORMAT_NUMBER(AVG(price), 2) as average_price 
    FROM p_sales 
    WHERE price >= 350000 
    GROUP BY view 
    ORDER BY view
""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04449868202209473 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')